In [1]:
# opengrid imports
from opengrid.library import misc, houseprint, caching
from opengrid.library.analysis import DailyAgg
from opengrid import config
from opengrid.library.slack import Slack
from opengrid.library import alerts
c=config.Config()

# other imports
import pandas as pd
import json
import charts
import numpy as np
import os
import datetime as dt
import pytz
BXL = pytz.timezone('Europe/Brussels')


# configuration for the plots
DEV = c.get('env', 'type') == 'dev' # DEV is True if we are in development environment, False if on the droplet
print("Environment configured for development: {}".format(DEV))
if not DEV:
    # production environment: don't try to display plots
    import matplotlib
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.dates import MinuteLocator, HourLocator, DateFormatter, AutoDateLocator, num2date

if DEV:
    if c.get('env', 'plots') == 'inline':
        %matplotlib inline
    else:
        %matplotlib qt
else:
    pass # don't try to render plots
plt.rcParams['figure.figsize'] = 12,8

Server running in the folder /usr/local/opengrid/notebooks/Analysis at 127.0.0.1:37903
Environment configured for development: True


In [2]:
hp = houseprint.Houseprint()
sensors = hp.get_sensors(sensortype='electricity') # sensor objects

# Remove some sensors
exclude = [
            '565de0a7dc64d8370aa321491217b85f' # 3E
          ]
solar = [x.key for x in hp.search_sensors(type='electricity', system='solar')]
exclude += solar

for s in sensors:
    if s.key in exclude:
        sensors.remove(s)

hp.init_tmpo()

Opening connection to Houseprint sheet
Opening spreadsheets
Parsing spreadsheets
34 Sites created
34 Devices created
97 sensors created
Houseprint parsing complete
Using tmpo database from /data/.tmpo/tmpo.sqlite3


In [3]:
hp.sync_tmpos()

100%|██████████| 97/97 [00:04<00:00, 22.81it/s]


In [4]:
sensors = hp.get_sensors() # sensor objects
t = []
for s in sensors:
    key = s.key
    lt = s.last_timestamp()
    if lt is None:
        continue
    nr_of_days = pd.Timestamp('now', tz='UTC') - lt
    t.append((key,nr_of_days))

In [5]:
df = pd.DataFrame(t).rename(columns={0: "sensor_id", 1: "timedelta"}).set_index('sensor_id')

In [6]:
df['seconds'] = df.timedelta.map(lambda x: x.total_seconds())
df['days'] = df.timedelta.map(lambda x: x.days)

In [7]:
df

,timedelta,seconds,days
sensor_id,,,
a6028457292f1866b468b63adcc82ee3,0 days 00:12:18.677498,7.386775e+02,0
53b1eb0479c83dee927fff10b0cb0fe6,0 days 00:03:46.690243,2.266902e+02,0
e8fa1a454e137aa36c1adfd9de94406b,0 days 01:20:34.709452,4.834709e+03,0
661684d35cd07dfc657a60b844860785,0 days 00:12:18.716784,7.387168e+02,0
e1dec16012edc11de998aeaa317274d7,0 days 00:03:46.724978,2.267250e+02,0
3aa40023a1683bfec0f01cce8d45fc70,0 days 00:03:46.732333,2.267323e+02,0
313b78fec4f845be91c328ee2f92c6d4,0 days 00:12:18.740420,7.387404e+02,0
b28509eb97137e723995838c393d49df,0 days 00:12:18.754326,7.387543e+02,0
99d7f090c9b191e31067a5fa922da025,0 days 00:03:46.763692,2.267637e+02,0


# Setup NoDataBot slack bot

In [8]:
slack_url = c.get('Slack', 'webhook')
username = 'NoDataBot'
channel = "junk" # we don't want to clutter up everything
emoji = ':warning:'
title = 'No data for sensor'
description = 'We have not found data for the last 24 hours (= 86400 seconds) for the following sensor. The result is expressed in seconds.'

slack = Slack(url=slack_url, username=username, channel=channel, emoji=emoji)

# Create the alerts and send

In [9]:
alerts.create_alerts(df, hp, 'no_sensor_data', slack, title, description, column='seconds')

/usr/local/opengrid/opengrid/library/alerts.py:24: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/opengrid/opengrid/config/alerts.cfg' mode='r' encoding='ANSI_X3.4-1968'>
  threshold = json.load(open(path_alerts, "r"))


In [ ]:
import pdb; pdb.pm()